In [1]:
from redcap import Project
import pandas as pd
import datetime

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = 'D6807C091B1FC14F57CC9B2972D0B398'
project = Project(api_url, api_key)


In [2]:
# import pandas as pd

# # Extract infant anthrops data
# inf_data = project.export_records(forms=['infant_anthropometry'])
# inf_data = pd.DataFrame(inf_data)

# inf_data = inf_data[['con_participantid_q1', 'ina_infantid', 'redcap_event_name', 'ian_date', 'ian_visitoutcome', 'ian_avglength', 'ian_avgweight', 'ian_avghc', 'ian_avgmuac', 'ian_triceps4', 'infant_anthropometry_complete']]
# df_infant = pd.DataFrame(inf_data)

# # Convert 'ian_date' to a date data type
# df_infant['ian_date'] = pd.to_datetime(df_infant['ian_date'], errors='coerce').dt.date

# # Filter rows where 'ian_date' is not null
# inf_anthrops = df_infant[df_infant['ian_date'].notnull()]

# # Extract required columns
# anthrops_df = pd.DataFrame({
#     'con_participantid_q1': df_infant['con_participantid_q1'],
#     'Infant ID': df_infant['ina_infantid'],
#     'redcap_event_name':df_infant['redcap_event_name'],
#     'Infant anthrops Date': pd.to_datetime(df_infant[df_infant['redcap_event_name'] == 'month_1_arm_2']['ian_date']),
#     'anthrops_m1': df_infant[df_infant['redcap_event_name'] == 'month_1_arm_2']['ian_avglength'],
#     'anthrops_m2': df_infant[df_infant['redcap_event_name'] == 'month_2_arm_2']['ian_avglength'],
#     'anthrops_m3': df_infant[df_infant['redcap_event_name'] == 'month_3_arm_2']['ian_avglength']
# })

# # Save the DataFrame to a CSV file
# anthrops_df.to_csv('Anthrops.csv', index=False)


In [3]:
import pandas as pd

# Extract infant anthrops data
inf_data = project.export_records(forms=['infant_anthropometry'])
inf_data = pd.DataFrame(inf_data)

inf_data = inf_data[['con_participantid_q1', 'ina_infantid', 'redcap_event_name', 'ian_date', 'ian_visitoutcome', 'ian_avglength', 'ian_avgweight', 'ian_avghc', 'ian_avgmuac', 'ian_triceps4', 'infant_anthropometry_complete']]
df_infant = pd.DataFrame(inf_data)

# Convert 'ian_date' to a date data type
df_infant['ian_date'] = pd.to_datetime(df_infant['ian_date'], errors='coerce').dt.date

# Filter rows where 'ian_date' is not null
inf_anthrops = df_infant[df_infant['ian_date'].notnull()]

# Pivot the DataFrame
pivot_df = inf_anthrops.pivot_table(
    index=['con_participantid_q1', 'ina_infantid'],
    columns='redcap_event_name',
    values='ian_avglength',
    aggfunc='first'
).reset_index()

# Rename columns for clarity
pivot_df.columns.name = None
pivot_df = pivot_df.rename(columns={
    'month_1_arm_2': 'anthrops_m1',
    'month_2_arm_2': 'anthrops_m2',
    'month_3_arm_2': 'anthrops_m3',
    'month_4_arm_2': 'anthrops_m4',
    'month_5_arm_2': 'anthrops_m5',
    'month_6_arm_2': 'anthrops_m6',
    'month_7_arm_2': 'anthrops_m7',
    'month_8_arm_2': 'anthrops_m8',
    'month_9_arm_2': 'anthrops_m9',
    'month_10_arm_2': 'anthrops_m10',
    'month_11_arm_2': 'anthrops_m11',
    'month_12_arm_2': 'anthrops_m12'
})

# Extract the date for the month 1 measurement
month1_date_info = inf_anthrops[inf_anthrops['redcap_event_name'] == 'month_1_arm_2'][['con_participantid_q1', 'ina_infantid', 'ian_date']].drop_duplicates()
month1_date_info = month1_date_info.rename(columns={'ian_date': 'Month 1 anthrops Date'})

# Merge the month 1 date information into the pivoted DataFrame
pivot_df = pivot_df.merge(month1_date_info, on=['con_participantid_q1', 'ina_infantid'], how='left')

# Save the DataFrame to a CSV file
pivot_df.to_csv('Anthrops.csv', index=False)
